* Given POI address, find the county and the town of the address

In [47]:
import json
import re

def countyMapping(mapping_list, addr):
    for county in mapping_list:
        match = re.search(county, addr)
        if match: return 1, county
    for county in mapping_list:
        match = re.search(county[:2], addr)
        if match: return 1, county
    #if addr has no county
    for county in mapping_list:
        pattern = "|".join(t for town in mapping_list[county] for t in mapping_list[county][town] if len(t)>2)
        if pattern == '': continue
        match = re.search(pattern, addr)
        if match: return 0, county
    return None, None

def townMapping(mapping_list, addr, countyflag):
    #if addr has no county
    if countyflag:
        addr = addr[2:]
    for key, value in mapping_list.items():
        pattern = "|".join(loc for loc in value)
        match = re.search(pattern, addr)
        if match: 
            return key

def POICountyTown(addr):
    with open("poi_data/taiwan_districts_mapping.json", 'r', encoding='utf-8-sig') as file:
        discrits = json.load(file)
    if addr == '' or len(addr) < 6:
            return None
    addr = re.sub(r'台', '臺', addr)
    countyflag, county = countyMapping(discrits, addr)
    town = townMapping(discrits[county], addr, countyflag) if county else None
    return county, town
    
    
test_addrs = ["桃園縣八德市建國路386號", "台南市永康區永科環路598號", "桃園縣龜山鄉文化一路55號", "花蓮縣974壽豐鄉牛山39之5號", '五峰鄉竹林村', '中華路二段445號']
for addr in test_addrs:
    print (POICountyTown(addr))


('桃園市', '八德區')
('臺南市', '永康區')
('桃園市', '龜山區')
('花蓮縣', '壽豐鄉')
('新竹縣', '五峰鄉')
(None, None)


* Calculate the distance of two (lat, lon)

In [25]:
from geopy.distance import great_circle, lonlat

In [53]:
bus_data = [{"RouteUID":"KHH100","RouteID":"100","RouteName":{"Zh_tw":"100","En":"100 "},"Operators":[{"OperatorID":"10270","OperatorName":{"Zh_tw":"統聯客運","En":"United Highway Bus Co., Ltd."},"OperatorCode":"UnitedHighwayBus","OperatorNo":"1201"}],"SubRouteUID":"KHH100","SubRouteID":"100","SubRouteName":{"Zh_tw":"100","En":"100 "},"Direction":0,"City":"Kaohsiung","CityCode":"KHH","Stops":[{"StopUID":"KHH178","StopID":"178","StopName":{"Zh_tw":"瑞豐站(瑞隆路)","En":"Rueifong Station (Rueilong Rd.)"},"StopBoarding":0,"StopSequence":1,"StopPosition":{"PositionLat":22.606123,"PositionLon":120.338443},"StationID":"03748","LocationCityCode":"KHH"},{"StopUID":"KHH308","StopID":"308","StopName":{"Zh_tw":"永豐路口","En":"Yongfong Rd. Intersection"},"StopBoarding":0,"StopSequence":2,"StopPosition":{"PositionLat":22.605742,"PositionLon":120.334387},"StationID":"01508","LocationCityCode":"KHH"},{"StopUID":"KHH831","StopID":"831","StopName":{"Zh_tw":"崗山市場","En":"Gangshan Market"},"StopBoarding":0,"StopSequence":3,"StopPosition":{"PositionLat":22.607333,"PositionLon":120.333533},"StationID":"03014","LocationCityCode":"KHH"},{"StopUID":"KHH832","StopID":"832","StopName":{"Zh_tw":"崗山北街口","En":"Gangshan N. St. Intersection"},"StopBoarding":0,"StopSequence":4,"StopPosition":{"PositionLat":22.609866,"PositionLon":120.33315},"StationID":"03012","LocationCityCode":"KHH"},{"StopUID":"KHH833","StopID":"833","StopName":{"Zh_tw":"鐵路機料廠","En":"Railway Material Plant"},"StopBoarding":0,"StopSequence":5,"StopPosition":{"PositionLat":22.612866,"PositionLon":120.332716},"StationID":"04673","LocationCityCode":"KHH"},{"StopUID":"KHH12822","StopID":"12822","StopName":{"Zh_tw":"鳳山工業區(監理所後)","En":"Fong Shan Industry Park (Behind the Motor Vehicle Office)"},"StopBoarding":0,"StopSequence":6,"StopPosition":{"PositionLat":22.614283,"PositionLon":120.332533},"StationID":"04158","LocationCityCode":"KHH"},{"StopUID":"KHH837","StopID":"837","StopName":{"Zh_tw":"鐵路機廠","En":"Railway Workshop"},"StopBoarding":0,"StopSequence":7,"StopPosition":{"PositionLat":22.617133,"PositionLon":120.33225},"StationID":"04676","LocationCityCode":"KHH"},{"StopUID":"KHH840","StopID":"840","StopName":{"Zh_tw":"武慶三路","En":"Wucing 3rd Rd."},"StopBoarding":0,"StopSequence":8,"StopPosition":{"PositionLat":22.619716,"PositionLon":120.331966},"StationID":"02122","LocationCityCode":"KHH"},{"StopUID":"KHH841","StopID":"841","StopName":{"Zh_tw":"五塊厝(武慶三路)","En":"Wukuaicuo  (Wucing 3rd Rd.)"},"StopBoarding":0,"StopSequence":9,"StopPosition":{"PositionLat":22.62145,"PositionLon":120.331783},"StationID":"00823","LocationCityCode":"KHH"},{"StopUID":"KHH140","StopID":"140","StopName":{"Zh_tw":"三信家商","En":"San Sin High School of Commerce and Home Economics"},"StopBoarding":0,"StopSequence":10,"StopPosition":{"PositionLat":22.62205,"PositionLon":120.329316},"StationID":"00196","LocationCityCode":"KHH"},{"StopUID":"KHH152","StopID":"152","StopName":{"Zh_tw":"國際商工","En":"Kuochi Senior Vocational High School of Commerce and Technology"},"StopBoarding":0,"StopSequence":11,"StopPosition":{"PositionLat":22.62115,"PositionLon":120.3265},"StationID":"02961","LocationCityCode":"KHH"},{"StopUID":"KHH52","StopID":"52","StopName":{"Zh_tw":"五權國小(三多二路)","En":"Wucyuan Elementry School (Sanduo  2nd Rd.  )"},"StopBoarding":0,"StopSequence":12,"StopPosition":{"PositionLat":22.620283,"PositionLon":120.323742},"StationID":"00836","LocationCityCode":"KHH"},{"StopUID":"KHH54","StopID":"54","StopName":{"Zh_tw":"和平路口","En":"Heping Rd. Intersection"},"StopBoarding":0,"StopSequence":13,"StopPosition":{"PositionLat":22.619547,"PositionLon":120.321486},"StationID":"01904","LocationCityCode":"KHH"},{"StopUID":"KHH59","StopID":"59","StopName":{"Zh_tw":"林德官(三和市場)","En":"Lindeguan (Sanhe Market)"},"StopBoarding":0,"StopSequence":14,"StopPosition":{"PositionLat":22.619017,"PositionLon":120.319886},"StationID":"02101","LocationCityCode":"KHH"},{"StopUID":"KHH843","StopID":"843","StopName":{"Zh_tw":"光華路口(光華夜市)","En":"Guanghua Rd. Intersection (Guanghua Night Market)"},"StopBoarding":0,"StopSequence":15,"StopPosition":{"PositionLat":22.618287,"PositionLon":120.317339},"StationID":"01596","LocationCityCode":"KHH"},{"StopUID":"KHH848","StopID":"848","StopName":{"Zh_tw":"三多市場(三多三路)","En":"Sanduo Market (Sanduo 3rd Rd.)"},"StopBoarding":0,"StopSequence":16,"StopPosition":{"PositionLat":22.617118,"PositionLon":120.313783},"StationID":"00182","LocationCityCode":"KHH"},{"StopUID":"KHH849","StopID":"849","StopName":{"Zh_tw":"三多復興路口","En":"Sanduo Fusing Rd. Intersection"},"StopBoarding":0,"StopSequence":17,"StopPosition":{"PositionLat":22.61655,"PositionLon":120.31235},"StationID":"00186","LocationCityCode":"KHH"},{"StopUID":"KHH854","StopID":"854","StopName":{"Zh_tw":"仁愛三街口","En":"Renai 3rd St. Intersection"},"StopBoarding":0,"StopSequence":18,"StopPosition":{"PositionLat":22.616033,"PositionLon":120.310733},"StationID":"00878","LocationCityCode":"KHH"},{"StopUID":"KHH855","StopID":"855","StopName":{"Zh_tw":"三多三路","En":"Sanduo 3rd Rd."},"StopBoarding":0,"StopSequence":19,"StopPosition":{"PositionLat":22.615446,"PositionLon":120.308815},"StationID":"00176","LocationCityCode":"KHH"},{"StopUID":"KHH17470","StopID":"17470","StopName":{"Zh_tw":"新光三越(捷運三多商圈站)","En":"Shin Kong Mitsukoshi Department Store (MRT Sandou Shopping District Station)"},"StopBoarding":0,"StopSequence":20,"StopPosition":{"PositionLat":22.614466,"PositionLon":120.306183},"StationID":"03535","LocationCityCode":"KHH"},{"StopUID":"KHH757","StopID":"757","StopName":{"Zh_tw":"大遠百百貨(捷運三多商圈站)","En":"Far Eastern Department Store (MRT Sandou Shopping District Station)"},"StopBoarding":0,"StopSequence":21,"StopPosition":{"PositionLat":22.613723,"PositionLon":120.303662},"StationID":"00450","LocationCityCode":"KHH"},{"StopUID":"KHH907","StopID":"907","StopName":{"Zh_tw":"自強三路口(三多四路)","En":"Zihciang 3rd Rd. Intersection (Sanduo 4th Rd.)"},"StopBoarding":0,"StopSequence":22,"StopPosition":{"PositionLat":22.612812,"PositionLon":120.300925},"StationID":"01743","LocationCityCode":"KHH"},{"StopUID":"KHH766","StopID":"766","StopName":{"Zh_tw":"八五大樓","En":"85 Sky Tower"},"StopBoarding":0,"StopSequence":23,"StopPosition":{"PositionLat":22.612362,"PositionLon":120.299587},"StationID":"00096","LocationCityCode":"KHH"},{"StopUID":"KHH859","StopID":"859","StopName":{"Zh_tw":"成功國小","En":"Chengkung Elementary School"},"StopBoarding":0,"StopSequence":24,"StopPosition":{"PositionLat":22.613875,"PositionLon":120.297828},"StationID":"01665","LocationCityCode":"KHH"},{"StopUID":"KHH865","StopID":"865","StopName":{"Zh_tw":"苓雅市場","En":"Lingya Market"},"StopBoarding":0,"StopSequence":25,"StopPosition":{"PositionLat":22.61696,"PositionLon":120.29676},"StationID":"02612","LocationCityCode":"KHH"},{"StopUID":"KHH80","StopID":"80","StopName":{"Zh_tw":"漢神百貨","En":"Hanshin Department Store"},"StopBoarding":0,"StopSequence":26,"StopPosition":{"PositionLat":22.619253,"PositionLon":120.295799},"StationID":"04028","LocationCityCode":"KHH"},{"StopUID":"KHH2595","StopID":"2595","StopName":{"Zh_tw":"大立百貨","En":"Talee Department Store"},"StopBoarding":0,"StopSequence":27,"StopPosition":{"PositionLat":22.621403,"PositionLon":120.296011},"StationID":"00281","LocationCityCode":"KHH"},{"StopUID":"KHH12287","StopID":"12287","StopName":{"Zh_tw":"大立百貨(文武二街口)","En":"Talee Department Store (Wunwu 2nd St. Intersection)"},"StopBoarding":0,"StopSequence":28,"StopPosition":{"PositionLat":22.621858,"PositionLon":120.297517},"StationID":"00283","LocationCityCode":"KHH"},{"StopUID":"KHH2622","StopID":"2622","StopName":{"Zh_tw":"城市光廊(捷運中央公園站)","En":"Urban Spotlight (MRT Central Park Station)"},"StopBoarding":0,"StopSequence":29,"StopPosition":{"PositionLat":22.622617,"PositionLon":120.299883},"StationID":"02407","LocationCityCode":"KHH"},{"StopUID":"KHH282","StopID":"282","StopName":{"Zh_tw":"捷運中央公園站","En":"MRT Central Park Station"},"StopBoarding":0,"StopSequence":30,"StopPosition":{"PositionLat":22.62468,"PositionLon":120.30125},"StationID":"03049","LocationCityCode":"KHH"},{"StopUID":"KHH300","StopID":"300","StopName":{"Zh_tw":"捷運美麗島站","En":"MRT Formosa Boulevard Station"},"StopBoarding":0,"StopSequence":31,"StopPosition":{"PositionLat":22.63014,"PositionLon":120.30193},"StationID":"03104","LocationCityCode":"KHH"},{"StopUID":"KHH332","StopID":"332","StopName":{"Zh_tw":"六合夜市(大港埔)","En":"Liouhe Tourist Night Market (Dagangpu)"},"StopBoarding":0,"StopSequence":32,"StopPosition":{"PositionLat":22.633844,"PositionLon":120.302261},"StationID":"00934","LocationCityCode":"KHH"},{"StopUID":"KHH674","StopID":"674","StopName":{"Zh_tw":"中山一路口","En":"Jhongshan 1st Rd. Intersection"},"StopBoarding":0,"StopSequence":33,"StopPosition":{"PositionLat":22.634416,"PositionLon":120.3036},"StationID":"00592","LocationCityCode":"KHH"},{"StopUID":"KHH11354","StopID":"11354","StopName":{"Zh_tw":"林森路口","En":"Linsen Rd. Intersection"},"StopBoarding":0,"StopSequence":34,"StopPosition":{"PositionLat":22.637686,"PositionLon":120.304302},"StationID":"02092","LocationCityCode":"KHH"},{"StopUID":"KHH2897","StopID":"2897","StopName":{"Zh_tw":"高雄火車站(捷運高雄車站)","En":"Kaohsiung Train Station (MRT Kaohsiung Main Station)"},"StopBoarding":0,"StopSequence":35,"StopPosition":{"PositionLat":22.636991,"PositionLon":120.302217},"StationID":"02845","LocationCityCode":"KHH"}],"UpdateTime":"2019-08-21T00:10:24+08:00","VersionID":182},{"RouteUID":"KHH100","RouteID":"100","RouteName":{"Zh_tw":"100","En":"100 "},"Operators":[{"OperatorID":"10270","OperatorName":{"Zh_tw":"統聯客運","En":"United Highway Bus Co., Ltd."},"OperatorCode":"UnitedHighwayBus","OperatorNo":"1201"}],"SubRouteUID":"KHH100","SubRouteID":"100","SubRouteName":{"Zh_tw":"100","En":"100 "},"Direction":1,"City":"Kaohsiung","CityCode":"KHH","Stops":[{"StopUID":"KHH19011","StopID":"19011","StopName":{"Zh_tw":"高雄火車站(捷運高雄車站)","En":"Kaohsiung Train Station (MRT Kaohsiung Main Station)"},"StopBoarding":0,"StopSequence":1,"StopPosition":{"PositionLat":22.636446,"PositionLon":120.30219},"StationID":"02845","LocationCityCode":"KHH"},{"StopUID":"KHH393","StopID":"393","StopName":{"Zh_tw":"六合夜市(大港埔)","En":"Liouhe Tourist Night Market (Dagangpu)"},"StopBoarding":0,"StopSequence":2,"StopPosition":{"PositionLat":22.633986,"PositionLon":120.30207},"StationID":"00934","LocationCityCode":"KHH"},{"StopUID":"KHH402","StopID":"402","StopName":{"Zh_tw":"捷運美麗島站","En":"MRT Formosa Boulevard Station"},"StopBoarding":0,"StopSequence":3,"StopPosition":{"PositionLat":22.6304,"PositionLon":120.30162},"StationID":"03103","LocationCityCode":"KHH"},{"StopUID":"KHH405","StopID":"405","StopName":{"Zh_tw":"捷運中央公園站","En":"MRT Central Park Station"},"StopBoarding":0,"StopSequence":4,"StopPosition":{"PositionLat":22.6247,"PositionLon":120.30103},"StationID":"03050","LocationCityCode":"KHH"},{"StopUID":"KHH825","StopID":"825","StopName":{"Zh_tw":"城市光廊(捷運中央公園站)","En":"Urban Spotlight (MRT Central Park Station)"},"StopBoarding":0,"StopSequence":5,"StopPosition":{"PositionLat":22.623033,"PositionLon":120.300356},"StationID":"02406","LocationCityCode":"KHH"},{"StopUID":"KHH12288","StopID":"12288","StopName":{"Zh_tw":"大立百貨(文武二街口)","En":"Talee Department Store (Wunwu 2nd St. Intersection)"},"StopBoarding":0,"StopSequence":6,"StopPosition":{"PositionLat":22.621986,"PositionLon":120.297313},"StationID":"00284","LocationCityCode":"KHH"},{"StopUID":"KHH91","StopID":"91","StopName":{"Zh_tw":"漢神百貨","En":"Hanshin Department Store"},"StopBoarding":0,"StopSequence":7,"StopPosition":{"PositionLat":22.619188,"PositionLon":120.295624},"StationID":"04029","LocationCityCode":"KHH"},{"StopUID":"KHH874","StopID":"874","StopName":{"Zh_tw":"苓雅市場","En":"Lingya Market"},"StopBoarding":0,"StopSequence":8,"StopPosition":{"PositionLat":22.616798,"PositionLon":120.296654},"StationID":"02611","LocationCityCode":"KHH"},{"StopUID":"KHH879","StopID":"879","StopName":{"Zh_tw":"成功國小","En":"Chengkung Elementary School"},"StopBoarding":0,"StopSequence":9,"StopPosition":{"PositionLat":22.613894,"PositionLon":120.297617},"StationID":"01666","LocationCityCode":"KHH"},{"StopUID":"KHH792","StopID":"792","StopName":{"Zh_tw":"八五大樓","En":"85 Sky Tower"},"StopBoarding":0,"StopSequence":10,"StopPosition":{"PositionLat":22.61242,"PositionLon":120.3},"StationID":"00097","LocationCityCode":"KHH"},{"StopUID":"KHH909","StopID":"909","StopName":{"Zh_tw":"自強三路口(三多四路)","En":"Zihciang 3rd Rd. Intersection (Sanduo 4th Rd.)"},"StopBoarding":0,"StopSequence":11,"StopPosition":{"PositionLat":22.612672,"PositionLon":120.300942},"StationID":"01742","LocationCityCode":"KHH"},{"StopUID":"KHH795","StopID":"795","StopName":{"Zh_tw":"大遠百百貨(捷運三多商圈站)","En":"Far Eastern Department Store (MRT Sandou Shopping District Station)"},"StopBoarding":0,"StopSequence":12,"StopPosition":{"PositionLat":22.613502,"PositionLon":120.303276},"StationID":"00449","LocationCityCode":"KHH"},{"StopUID":"KHH17277","StopID":"17277","StopName":{"Zh_tw":"新光三越(捷運三多商圈站)","En":"Shin Kong Mitsukoshi Department Store (MRT Sandou Shopping District Station)"},"StopBoarding":0,"StopSequence":13,"StopPosition":{"PositionLat":22.614464,"PositionLon":120.306187},"StationID":"03536","LocationCityCode":"KHH"},{"StopUID":"KHH885","StopID":"885","StopName":{"Zh_tw":"三多三路","En":"Sanduo 3rd Rd."},"StopBoarding":0,"StopSequence":14,"StopPosition":{"PositionLat":22.615385,"PositionLon":120.309006},"StationID":"00177","LocationCityCode":"KHH"},{"StopUID":"KHH886","StopID":"886","StopName":{"Zh_tw":"仁愛三街口","En":"Renai 3rd St. Intersection"},"StopBoarding":0,"StopSequence":15,"StopPosition":{"PositionLat":22.615933,"PositionLon":120.310733},"StationID":"00877","LocationCityCode":"KHH"},{"StopUID":"KHH892","StopID":"892","StopName":{"Zh_tw":"三多復興路口","En":"Sanduo Fusing Rd. Intersection"},"StopBoarding":0,"StopSequence":16,"StopPosition":{"PositionLat":22.61645,"PositionLon":120.31235},"StationID":"00187","LocationCityCode":"KHH"},{"StopUID":"KHH893","StopID":"893","StopName":{"Zh_tw":"三多市場(三多三路)","En":"Sanduo Market (Sanduo 3rd Rd.)"},"StopBoarding":0,"StopSequence":17,"StopPosition":{"PositionLat":22.61735,"PositionLon":120.314883},"StationID":"00183","LocationCityCode":"KHH"},{"StopUID":"KHH117","StopID":"117","StopName":{"Zh_tw":"光華路口(光華夜市)","En":"Guanghua Rd. Intersection (Guanghua Night Market)"},"StopBoarding":0,"StopSequence":18,"StopPosition":{"PositionLat":22.618416,"PositionLon":120.318233},"StationID":"01597","LocationCityCode":"KHH"},{"StopUID":"KHH125","StopID":"125","StopName":{"Zh_tw":"林德官(三和市場)","En":"Lindeguan (Sanhe Market)"},"StopBoarding":0,"StopSequence":19,"StopPosition":{"PositionLat":22.618872,"PositionLon":120.319796},"StationID":"02102","LocationCityCode":"KHH"},{"StopUID":"KHH126","StopID":"126","StopName":{"Zh_tw":"和平路口","En":"Heping Rd. Intersection"},"StopBoarding":0,"StopSequence":20,"StopPosition":{"PositionLat":22.619303,"PositionLon":120.321102},"StationID":"01905","LocationCityCode":"KHH"},{"StopUID":"KHH132","StopID":"132","StopName":{"Zh_tw":"五權國小(三多二路)","En":"Wucyuan Elementry School (Sanduo  2nd Rd.  )"},"StopBoarding":0,"StopSequence":21,"StopPosition":{"PositionLat":22.620194,"PositionLon":120.323886},"StationID":"00835","LocationCityCode":"KHH"},{"StopUID":"KHH555","StopID":"555","StopName":{"Zh_tw":"國際商工","En":"Kuochi Senior Vocational High School of Commerce and Technology"},"StopBoarding":0,"StopSequence":22,"StopPosition":{"PositionLat":22.62105,"PositionLon":120.3265},"StationID":"02962","LocationCityCode":"KHH"},{"StopUID":"KHH569","StopID":"569","StopName":{"Zh_tw":"三信家商","En":"San Sin High School of Commerce and Home Economics"},"StopBoarding":0,"StopSequence":23,"StopPosition":{"PositionLat":22.621949999999998,"PositionLon":120.329316},"StationID":"00197","LocationCityCode":"KHH"},{"StopUID":"KHH896","StopID":"896","StopName":{"Zh_tw":"五塊厝(武慶三路)","En":"Wukuaicuo  (Wucing 3rd Rd.)"},"StopBoarding":0,"StopSequence":24,"StopPosition":{"PositionLat":22.62145,"PositionLon":120.331683},"StationID":"00824","LocationCityCode":"KHH"},{"StopUID":"KHH897","StopID":"897","StopName":{"Zh_tw":"武慶三路","En":"Wucing 3rd Rd."},"StopBoarding":0,"StopSequence":25,"StopPosition":{"PositionLat":22.619716,"PositionLon":120.331866},"StationID":"02121","LocationCityCode":"KHH"},{"StopUID":"KHH899","StopID":"899","StopName":{"Zh_tw":"鐵路機廠","En":"Railway Workshop"},"StopBoarding":0,"StopSequence":26,"StopPosition":{"PositionLat":22.616978,"PositionLon":120.332133},"StationID":"04675","LocationCityCode":"KHH"},{"StopUID":"KHH12823","StopID":"12823","StopName":{"Zh_tw":"鳳山工業區(監理所後)","En":"Fong Shan Industry Park (Behind the Motor Vehicle Office)"},"StopBoarding":0,"StopSequence":27,"StopPosition":{"PositionLat":22.614653,"PositionLon":120.332408},"StationID":"04159","LocationCityCode":"KHH"},{"StopUID":"KHH903","StopID":"903","StopName":{"Zh_tw":"鐵路機料廠","En":"Railway Material Plant"},"StopBoarding":0,"StopSequence":28,"StopPosition":{"PositionLat":22.612681,"PositionLon":120.332553},"StationID":"04674","LocationCityCode":"KHH"},{"StopUID":"KHH905","StopID":"905","StopName":{"Zh_tw":"崗山北街口","En":"Gangshan N. St. Intersection"},"StopBoarding":0,"StopSequence":29,"StopPosition":{"PositionLat":22.60985,"PositionLon":120.333022},"StationID":"03013","LocationCityCode":"KHH"},{"StopUID":"KHH906","StopID":"906","StopName":{"Zh_tw":"崗山市場","En":"Gangshan Market"},"StopBoarding":0,"StopSequence":30,"StopPosition":{"PositionLat":22.607278,"PositionLon":120.333356},"StationID":"03015","LocationCityCode":"KHH"},{"StopUID":"KHH384","StopID":"384","StopName":{"Zh_tw":"永豐路口","En":"Yongfong Rd. Intersection"},"StopBoarding":0,"StopSequence":31,"StopPosition":{"PositionLat":22.605583,"PositionLon":120.334383},"StationID":"01509","LocationCityCode":"KHH"},{"StopUID":"KHH288","StopID":"288","StopName":{"Zh_tw":"瑞豐站(瑞隆路)","En":"Rueifong Station (Rueilong Rd.)"},"StopBoarding":0,"StopSequence":32,"StopPosition":{"PositionLat":22.606123,"PositionLon":120.338443},"StationID":"03749","LocationCityCode":"KHH"}],"UpdateTime":"2019-08-21T00:10:24+08:00","VersionID":182},{"RouteUID":"KHH11","RouteID":"11","RouteName":{"Zh_tw":"11","En":"11 "},"Operators":[{"OperatorID":"10270","OperatorName":{"Zh_tw":"統聯客運","En":"United Highway Bus Co., Ltd."},"OperatorCode":"UnitedHighwayBus","OperatorNo":"1201"}],"SubRouteUID":"KHH11","SubRouteID":"11","SubRouteName":{"Zh_tw":"11","En":"11 "},"Direction":0,"City":"Kaohsiung","CityCode":"KHH","Stops":[{"StopUID":"KHH32","StopID":"32","StopName":{"Zh_tw":"瑞豐站(武營路)","En":"Rueifong Station (Wuying Rd.)"},"StopBoarding":0,"StopSequence":1,"StopPosition":{"PositionLat":22.60686,"PositionLon":120.337999},"StationID":"03746","LocationCityCode":"KHH"},{"StopUID":"KHH35","StopID":"35","StopName":{"Zh_tw":"北極殿(鳳山)","En":"Beiji Temple (Fongshan)"},"StopBoarding":0,"StopSequence":2,"StopPosition":{"PositionLat":22.610908,"PositionLon":120.337397},"StationID":"01227","LocationCityCode":"KHH"},{"StopUID":"KHH38","StopID":"38","StopName":{"Zh_tw":"紫竹林精舍","En":"Violet Bamboo Grove Buddhist Temple"},"StopBoarding":0,"StopSequence":3,"StopPosition":{"PositionLat":22.612322,"PositionLon":120.337206},"StationID":"03396","LocationCityCode":"KHH"},{"StopUID":"KHH11300","StopID":"11300","StopName":{"Zh_tw":"新甲國小","En":"Hsin Chya Elementary School"},"StopBoarding":0,"StopSequence":4,"StopPosition":{"PositionLat":22.612797,"PositionLon":120.340835},"StationID":"03531","LocationCityCode":"KHH"},{"StopUID":"KHH86","StopID":"86","StopName":{"Zh_tw":"鳳新高中","En":"Feng-Hsin Senior High School"},"StopBoarding":0,"StopSequence":5,"StopPosition":{"PositionLat":22.612895,"PositionLon":120.343272},"StationID":"04235","LocationCityCode":"KHH"},{"StopUID":"KHH13853","StopID":"13853","StopName":{"Zh_tw":"鳳新高中(南京路)","En":"Feng-Hsin Senior High School (Nanjing Rd.)"},"StopBoarding":0,"StopSequence":6,"StopPosition":{"PositionLat":22.61334,"PositionLon":120.34567},"StationID":"04236","LocationCityCode":"KHH"},{"StopUID":"KHH11301","StopID":"11301","StopName":{"Zh_tw":"新康街口","En":"Sinkang St. Intersection"},"StopBoarding":0,"StopSequence":7,"StopPosition":{"PositionLat":22.61716,"PositionLon":120.34313},"StationID":"03589","LocationCityCode":"KHH"},{"StopUID":"KHH11302","StopID":"11302","StopName":{"Zh_tw":"新強路口","En":"Sinciang Rd. Intersection"},"StopBoarding":0,"StopSequence":8,"StopPosition":{"PositionLat":22.61671,"PositionLon":120.339296},"StationID":"03590","LocationCityCode":"KHH"},{"StopUID":"KHH42","StopID":"42","StopName":{"Zh_tw":"武昌路口(武營路)","En":"Wuchang Rd. Intersection (Wuying Rd.)"},"StopBoarding":0,"StopSequence":9,"StopPosition":{"PositionLat":22.618062,"PositionLon":120.33645},"StationID":"02113","LocationCityCode":"KHH"},{"StopUID":"KHH43","StopID":"43","StopName":{"Zh_tw":"武營路","En":"Wuying Rd."},"StopBoarding":0,"StopSequence":10,"StopPosition":{"PositionLat":22.620369,"PositionLon":120.336142},"StationID":"02126","LocationCityCode":"KHH"},{"StopUID":"KHH45","StopID":"45","StopName":{"Zh_tw":"三多一路口","En":"Sanduo 1st Rd. Intersection"},"StopBoarding":0,"StopSequence":11,"StopPosition":{"PositionLat":22.622739,"PositionLon":120.335823},"StationID":"00169","LocationCityCode":"KHH"},{"StopUID":"KHH100","StopID":"100","StopName":{"Zh_tw":"輔仁路口(三多一路)","En":"Furen Rd. Intersection (Sanduo 1st Rd.)"},"StopBoarding":0,"StopSequence":12,"StopPosition":{"PositionLat":22.623697,"PositionLon":120.33391},"StationID":"04121","LocationCityCode":"KHH"},{"StopUID":"KHH119","StopID":"119","StopName":{"Zh_tw":"五塊厝(三多一路)","En":"Wukuaicuo  (Sanduo 1st Rd. )"},"StopBoarding":0,"StopSequence":13,"StopPosition":{"PositionLat":22.623217,"PositionLon":120.332533},"StationID":"00822","LocationCityCode":"KHH"},{"StopUID":"KHH139","StopID":"139","StopName":{"Zh_tw":"三信家商","En":"San Sin High School of Commerce and Home Economics"},"StopBoarding":0,"StopSequence":14,"StopPosition":{"PositionLat":22.622183,"PositionLon":120.329464},"StationID":"00196","LocationCityCode":"KHH"},{"StopUID":"KHH154","StopID":"154","StopName":{"Zh_tw":"國際商工","En":"Kuochi Senior Vocational High School of Commerce and Technology"},"StopBoarding":0,"StopSequence":15,"StopPosition":{"PositionLat":22.621211,"PositionLon":120.326536},"StationID":"02961","LocationCityCode":"KHH"},{"StopUID":"KHH50","StopID":"50","StopName":{"Zh_tw":"五權國小(三多二路)","En":"Wucyuan Elementry School (Sanduo  2nd Rd.  )"},"StopBoarding":0,"StopSequence":16,"StopPosition":{"PositionLat":22.620235,"PositionLon":120.323719},"StationID":"00836","LocationCityCode":"KHH"},{"StopUID":"KHH55","StopID":"55","StopName":{"Zh_tw":"和平路口","En":"Heping Rd. Intersection"},"StopBoarding":0,"StopSequence":17,"StopPosition":{"PositionLat":22.619547,"PositionLon":120.321486},"StationID":"01904","LocationCityCode":"KHH"},{"StopUID":"KHH58","StopID":"58","StopName":{"Zh_tw":"林德官(三和市場)","En":"Lindeguan (Sanhe Market)"},"StopBoarding":0,"StopSequence":18,"StopPosition":{"PositionLat":22.619017,"PositionLon":120.319886},"StationID":"02101","LocationCityCode":"KHH"},{"StopUID":"KHH62","StopID":"62","StopName":{"Zh_tw":"三多二路口(光華夜市)","En":"Sanduo 2nd Rd. Intersection (Guanghua Night Market)"},"StopBoarding":0,"StopSequence":19,"StopPosition":{"PositionLat":22.618933,"PositionLon":120.317534},"StationID":"00175","LocationCityCode":"KHH"},{"StopUID":"KHH63","StopID":"63","StopName":{"Zh_tw":"四維路口(光華一路)","En":"Sihwei Rd. Intersection (Guanghua 1st Rd.)"},"StopBoarding":0,"StopSequence":20,"StopPosition":{"PositionLat":22.620756,"PositionLon":120.316842},"StationID":"01307","LocationCityCode":"KHH"},{"StopUID":"KHH65","StopID":"65","StopName":{"Zh_tw":"苓雅一路口","En":"Lingya 1st Rd. Intersection"},"StopBoarding":0,"StopSequence":21,"StopPosition":{"PositionLat":22.622529,"PositionLon":120.316251},"StationID":"02610","LocationCityCode":"KHH"},{"StopUID":"KHH67","StopID":"67","StopName":{"Zh_tw":"青年路口(光華一路)","En":"Cingnian Rd. Intersection (Guanghua 1st Rd.)"},"StopBoarding":0,"StopSequence":22,"StopPosition":{"PositionLat":22.623475,"PositionLon":120.315881},"StationID":"02246","LocationCityCode":"KHH"},{"StopUID":"KHH71","StopID":"71","StopName":{"Zh_tw":"四維國小(青年一路)","En":"Sih-Wei Primary School (Cingnian 1st Rd.)"},"StopBoarding":0,"StopSequence":23,"StopPosition":{"PositionLat":22.623803,"PositionLon":120.313397},"StationID":"01306","LocationCityCode":"KHH"},{"StopUID":"KHH72","StopID":"72","StopName":{"Zh_tw":"民權路口(青年一路)","En":"Minciuan Rd. Intersection (Cingnian 1st Rd.)"},"StopBoarding":0,"StopSequence":24,"StopPosition":{"PositionLat":22.623125,"PositionLon":120.311319},"StationID":"01473","LocationCityCode":"KHH"},{"StopUID":"KHH73","StopID":"73","StopName":{"Zh_tw":"國民市場","En":"Guomin Market"},"StopBoarding":0,"StopSequence":25,"StopPosition":{"PositionLat":22.622167,"PositionLon":120.308514},"StationID":"02933","LocationCityCode":"KHH"},{"StopUID":"KHH74","StopID":"74","StopName":{"Zh_tw":"青年一路","En":"Cingnian 1st Rd."},"StopBoarding":0,"StopSequence":26,"StopPosition":{"PositionLat":22.621539,"PositionLon":120.306656},"StationID":"02243","LocationCityCode":"KHH"},{"StopUID":"KHH75","StopID":"75","StopName":{"Zh_tw":"仁智街口","En":"Renjhih St. Intersection"},"StopBoarding":0,"StopSequence":27,"StopPosition":{"PositionLat":22.620403,"PositionLon":120.303144},"StationID":"00872","LocationCityCode":"KHH"},{"StopUID":"KHH76","StopID":"76","StopName":{"Zh_tw":"仁德街口(青年二路)","En":"Rende St. Intersection (Cingnian 2nd Rd.)"},"StopBoarding":0,"StopSequence":28,"StopPosition":{"PositionLat":22.619644,"PositionLon":120.300883},"StationID":"00892","LocationCityCode":"KHH"},{"StopUID":"KHH77","StopID":"77","StopName":{"Zh_tw":"青年二路","En":"Cingnian 2nd Rd."},"StopBoarding":0,"StopSequence":29,"StopPosition":{"PositionLat":22.618658,"PositionLon":120.298314},"StationID":"02244","LocationCityCode":"KHH"},{"StopUID":"KHH78","StopID":"78","StopName":{"Zh_tw":"漢神百貨","En":"Hanshin Department Store"},"StopBoarding":0,"StopSequence":30,"StopPosition":{"PositionLat":22.61913,"PositionLon":120.29591},"StationID":"04028","LocationCityCode":"KHH"},{"StopUID":"KHH928","StopID":"928","StopName":{"Zh_tw":"高雄女中(真愛碼頭)","En":"Kaohsiung Municipal Girls` Senior High School (Love Pier)"},"StopBoarding":0,"StopSequence":31,"StopPosition":{"PositionLat":22.620799,"PositionLon":120.2922},"StationID":"02828","LocationCityCode":"KHH"},{"StopUID":"KHH933","StopID":"933","StopName":{"Zh_tw":"大成街口(輕軌真愛碼頭站)","En":"Dacheng St. Intersection (LRT Love Pier Station）"},"StopBoarding":0,"StopSequence":32,"StopPosition":{"PositionLat":22.62125,"PositionLon":120.28853},"StationID":"00295","LocationCityCode":"KHH"},{"StopUID":"KHH945","StopID":"945","StopName":{"Zh_tw":"大智路口(五福四路)","En":"Dajhih Rd. Intersection (Wufu 4th Rd.)"},"StopBoarding":0,"StopSequence":33,"StopPosition":{"PositionLat":22.621742,"PositionLon":120.287431},"StationID":"00396","LocationCityCode":"KHH"},{"StopUID":"KHH2962","StopID":"2962","StopName":{"Zh_tw":"大智路(光榮國小)","En":"Dajhih Rd. (Guangrong Elementary School)"},"StopBoarding":0,"StopSequence":34,"StopPosition":{"PositionLat":22.623681,"PositionLon":120.286683},"StationID":"00392","LocationCityCode":"KHH"},{"StopUID":"KHH2658","StopID":"2658","StopName":{"Zh_tw":"捷運鹽埕埔站(大仁路)","En":"MRT Yanchengpu Station (Daren Rd.)"},"StopBoarding":0,"StopSequence":35,"StopPosition":{"PositionLat":22.625203,"PositionLon":120.285357},"StationID":"03129","LocationCityCode":"KHH"}],"UpdateTime":"2019-08-21T00:10:24+08:00","VersionID":182}]

with open("poi_data/scenic_spot_C_f.json", 'r', encoding='utf-8-sig') as file:
    data = json.load(file)
    i = 0
    scenespot_data = []
    for info in data['XML_Head']['Infos']['Info']:
        scenespot_data.append(info)
        i += 1
        if i == 4:
            data = None
            break

#given the stop imformations, return distance in kilometers
def stopsDistance(stop1, stop2):
    pos1 = (stop1['StopPosition']['PositionLat'], stop1['StopPosition']['PositionLon'])
    pos2 = (stop2['StopPosition']['PositionLat'], stop2['StopPosition']['PositionLon'])
    return (great_circle(pos1, pos2).km)

#given the POI and stop information, return distance in kilometers
def POIstopDistance(poi, stop):
    if poi['Px'] and poi['Py'] != "":
        pos1 = (stop['StopPosition']['PositionLat'], stop['StopPosition']['PositionLon'])
        pos2 = (poi['Py'], poi['Px'])
        return (great_circle(pos1, pos2).km)

#given the lontitudes and latitudes, return distance in kilometers
def returnDistance(lat1, lon1, lat2, lon2):
    if lon1 and  lat1 and lon2 and lat2:
        return (great_circle((lat1, lon1), (lat2, lon2)).km)

In [27]:
target_stop = {'LocationCityCode': 'KHH',
                    'StationID': '00295',
                    'StopBoarding': 0,
                    'StopID': '933',
                    'StopName': {'En': 'Dacheng St. Intersection (LRT Love Pier Station）',
                                 'Zh_tw': '大成街口(輕軌真愛碼頭站)'},
                    'StopPosition': {'PositionLat': 22.62125, 'PositionLon': 120.28853},
                    'StopSequence': 32,
                    'StopUID': 'KHH933'}

In [28]:
print ("Test distance between stops")
for line in bus_data:
    print (line['RouteUID'])
    for stop in line['Stops']:
        print ("{} {}: {}".format(target_stop['StopName']['Zh_tw'], stop['StopName']['Zh_tw'], stopsDistance(target_stop, stop)))

Test distance between stops
KHH100
大成街口(輕軌真愛碼頭站) 瑞豐站(瑞隆路): 5.392431664296202
大成街口(輕軌真愛碼頭站) 永豐路口: 5.012979509461742
大成街口(輕軌真愛碼頭站) 崗山市場: 4.871685666285807
大成街口(輕軌真愛碼頭站) 崗山北街口: 4.751723455978149
大成街口(輕軌真愛碼頭站) 鐵路機料廠: 4.630237250814675
大成街口(輕軌真愛碼頭站) 鳳山工業區(監理所後): 4.58256491567212
大成街口(輕軌真愛碼頭站) 鐵路機廠: 4.510803514182449
大成街口(輕軌真愛碼頭站) 武慶三路: 4.4615829996575
大成街口(輕軌真愛碼頭站) 五塊厝(武慶三路): 4.439565544341196
大成街口(輕軌真愛碼頭站) 三信家商: 4.187231605377393
大成街口(輕軌真愛碼頭站) 國際商工: 3.897280163048769
大成街口(輕軌真愛碼頭站) 五權國小(三多二路): 3.6157920098215177
大成街口(輕軌真愛碼頭站) 和平路口: 3.3879402775156655
大成街口(輕軌真愛碼頭站) 林德官(三和市場): 3.22798801285438
大成街口(輕軌真愛碼頭站) 光華路口(光華夜市): 2.9753026815268426
大成街口(輕軌真愛碼頭站) 三多市場(三多三路): 2.632428055726105
大成街口(輕軌真愛碼頭站) 三多復興路口: 2.5001724283860525
大成街口(輕軌真愛碼頭站) 仁愛三街口: 2.351645212232098
大成街口(輕軌真愛碼頭站) 三多三路: 2.1798359646379573
大成街口(輕軌真愛碼頭站) 新光三越(捷運三多商圈站): 1.962711152882836
大成街口(輕軌真愛碼頭站) 大遠百百貨(捷運三多商圈站): 1.7643527598990234
大成街口(輕軌真愛碼頭站) 自強三路口(三多四路): 1.5808251233345132
大成街口(輕軌真愛碼頭站) 八五大樓: 1.5049301987276575
大成街口(輕軌真愛碼頭站) 成功國

In [29]:
print ("Test distance between stop and POI")
for poi in scenespot_data:
    print ("{} {}: {}".format(target_stop['StopName']['Zh_tw'], poi['Name'], stopsDistance(target_stop, stop)))

Test distance between stop and POI
大成街口(輕軌真愛碼頭站) 宏亞食品巧克力觀光工廠: 0.5470569841111657
大成街口(輕軌真愛碼頭站) 台灣金屬創意館: 0.5470569841111657
大成街口(輕軌真愛碼頭站) 臺灣菸酒(股)公司林口觀光酒廠: 0.5470569841111657
大成街口(輕軌真愛碼頭站) 水璉、牛山海岸: 0.5470569841111657


* Related busses and stops
* Nearest stops

In [103]:
import json
import numpy as np

#RouteUID
target_route = 'KHH1111'
#StationID
target_stop = '03758'

#given routeUID, return all stops in one direction
def allStops(routeUID, file):
    stops = []
    for route in file:
        for stop in route["Stops"]:
            if route['RouteUID'] == routeUID:
                stops.append(stop)
        if stops: return stops

#given stopName Zh_tw, return related route
def relatedRoutes(StopName, file):
    return [route['RouteUID'] for route in kaohsiung_bus for stop in route["Stops"] if stop['StopName']['Zh_tw'] == StopName]

#given the lat, lon, city, and num of return stops, return the n nearest stops
def nearestStops(lat, lon, city, n=5):
    county_file_map = {'臺北市': 'Taipei_route.json','基隆市': 'Keelung_route.json','新北市': 'NewTaipei_route.json','連江縣': 'LienchiangCounty.json','宜蘭縣': 'YilanCounty_route.json','新竹市': 'Hsinchu_route.json','新竹縣': 'HsinchuCounty_route.json','桃園市': 'Taoyuan.json','苗栗縣': 'MiaoliCounty_route.json','臺中市': 'Taichung.json','彰化縣': 'ChanghuaCounty_route.json','南投縣': 'NantouCounty_route.json','嘉義市': 'Chiayi_route.json','嘉義縣': 'ChiayiCounty_route.json','雲林縣': 'YunlinCounty_route.json','臺南市': 'Tainan_route.json','高雄市': 'Kaohsiung.json','澎湖縣': 'PenghuCounty_route.json','金門縣': 'KinmenCounty_route.json','屏東縣': 'PingtungCounty_route.json','臺東縣': 'TaitungCounty_route.json','花蓮縣': 'HualienCounty_route.json'}
    with open('Transportation_data/bus_schedule/'+county_file_map[city]) as file:
        bus_file = json.load(file)
        
    min_dis = np.array([np.finfo(np.float32).max for i in range(n)])
    min_stops = np.array(["_____" for i in range(n)])
    for route in bus_file:
        for stop in allStops(route['RouteUID'], bus_file):
            dis = returnDistance(lat, lon, stop['StopPosition']['PositionLat'], stop['StopPosition']['PositionLon'])
            #find if the distance is smaller than element of min_dis
            for tmp in dis < min_dis:
                if tmp and stop['StopName']['Zh_tw'] not in min_stops:
#                     print ("dis: {}, ID: {}, Name: {}, max_id: {}".format(dis, stop['StationID'], stop['StopName']['Zh_tw'], np.argmax(min_dis)))
                    min_stops[np.argmax(min_dis)] = stop['StopName']['Zh_tw']
                    min_dis[np.argmax(min_dis)] = float(dis)
                    break
    return [{min_stops[idx]: min_dis[idx]} for idx in range(n)]
    
test_poi = {"Name": "烏來瀑布 空中纜車", "Add": "新北市烏來區瀑布路1-1號", "Px": 121.551590, "Py": 24.849040} 
city, town = POICountyTown(test_poi["Add"])
stops = nearestStops(test_poi["Py"], test_poi['Px'], city)
print (stops)                               
# print (allStops(target_route, kaohsiung_bus))
# print (relatedRoute(target_stop, kaohsiung_bus))

[{'污水處理廠': 3.070026}, {'烏來': 1.9504124}, {'巨龍山莊': 2.8400834}, {'觀光大橋': 2.2414105}, {'堰堤': 2.505791}]


* Find shortest path??

In [90]:
#given pois addr, return transportation path between pois, consider only bus now
def transportationPath(poi1, poi2):
    city1, town1 = POICountyTown(poi1["Add"])
    stops1 = nearestStops(poi1["Py"], poi1['Px'], city1)
    city2, town2 = POICountyTown(poi2["Add"])
    stops2 = nearestStops(poi2["Py"], poi2['Px'], city2)
    
???    

* StationID and StopNamt mapping

In [143]:
with open('Transportation_data/bus_schedule/Taipei_route.json') as file:
    bus_file = json.load(file)

#stationID mapping to stop name
station_map = {}
for route in bus_file:
    for stop in route['Stops']:
        if stop['StopName']['Zh_tw'] not in station_map:
            station_map[stop['StopName']['Zh_tw']] = []
        if stop['StationID'] not in station_map[stop['StopName']['Zh_tw']]:
            station_map[stop['StopName']['Zh_tw']].append(stop['StationID'])
                                                      
station_map

{'歡仔園': ['2717', '1000431'],
 '僑中一街': ['6005', '2684'],
 '大觀路一段28巷': ['1165603306', '6006'],
 '大觀國小': ['4697', '4696'],
 '國立臺灣藝術大學': ['2652', '60019'],
 '中山實小': ['6007', '14502'],
 '板橋榮家': ['6009', '2626'],
 '力行新村': ['6010', '14505', '9443'],
 '南雅站': ['2632', '1000661'],
 '亞東技術學院': ['2601', '2602'],
 '豫章工商': ['2702', '6011'],
 '仁愛新村': ['2562', '2563'],
 '福星里': ['2666', '2691'],
 '鄉雲里': ['6012', '6013'],
 '後站商圈': ['2636', '2637'],
 '介壽公園': ['1001063'],
 '民權路口': ['6002', '14207', '15047', '15048'],
 '新北市政府(新府路)': ['2699', '2700'],
 '板橋公車站': ['70717', '70665', '70716'],
 '新北市政府(中山路)': ['6200', '6199'],
 '中山漢生路口': ['2606', '6198'],
 '西安里': ['2594', '14201', '3542'],
 '中山民生路口': ['1165603235', '1165603220'],
 '埔墘國小': ['2643', '14219'],
 '埔墘(中山路)': ['14231', '14220'],
 '埔墘派出所': ['2642', '6195'],
 '光復橋': ['14221', '2585'],
 '西園路二段': ['4629', '1756'],
 '華江高中(西園)': ['4627', '1804'],
 '西園29服飾基地': ['4628', '1825'],
 '大理服飾': ['4626', '1738'],
 '捷運龍山寺站': ['1794', '3912', '3911'],
 '龍山寺(康定)': ['10009

* Output bus graph, ex: Taipei

`graph =
{
    'a':['b', 'c']
    'b':['a']
    'c':['a']
}`

In [140]:
with open('Transportation_data/bus_schedule/Taipei_route.json') as file:
    bus_file = json.load(file)
    
#generate empty graph dict
graph = {}  
for route in bus_file:
    for stop in route['Stops']:
        if stop['StopName']['Zh_tw'] not in graph:
            graph[stop['StopName']['Zh_tw']] = []
    
for route in bus_file:
    stops = route['Stops']
    for index in range(len(stops)):
        if len(stops) < 2: break
        if index == 0 :
            if stops[index+1]['StopName']['Zh_tw'] not in graph[stops[index]['StopName']['Zh_tw']]:
                graph[stops[index]['StopName']['Zh_tw']].append(stops[index+1]['StopName']['Zh_tw'])
        elif index == len(route['Stops'])-1:
            if stops[index-1]['StopName']['Zh_tw'] not in graph[stops[index]['StopName']['Zh_tw']]:
                graph[stops[index]['StopName']['Zh_tw']].append(stops[index-1]['StopName']['Zh_tw'])
        else:
            if stops[index-1]['StopName']['Zh_tw'] not in graph[stops[index]['StopName']['Zh_tw']]:
                graph[stops[index]['StopName']['Zh_tw']].append(stops[index-1]['StopName']['Zh_tw'])
            if stops[index+1]['StopName']['Zh_tw'] not in graph[stops[index]['StopName']['Zh_tw']]:
                graph[stops[index]['StopName']['Zh_tw']].append(stops[index+1]['StopName']['Zh_tw'])
        
graph        

{'歡仔園': ['僑中一街'],
 '僑中一街': ['歡仔園', '大觀路一段28巷', '大觀路28巷', '佳佳幼稚園'],
 '大觀路一段28巷': ['僑中一街', '大觀國小'],
 '大觀國小': ['大觀路一段28巷', '國立臺灣藝術大學', '接雲寺'],
 '國立臺灣藝術大學': ['大觀國小', '中山實小', '臨時站'],
 '中山實小': ['國立臺灣藝術大學', '板橋榮家'],
 '板橋榮家': ['中山實小', '力行新村'],
 '力行新村': ['板橋榮家', '南雅站', '福安里', '圓山新城一', '圓山新城二'],
 '南雅站': ['力行新村', '亞東技術學院', '浮洲橋', '中央一站', '中央路'],
 '亞東技術學院': ['南雅站', '豫章工商', '信義路'],
 '豫章工商': ['亞東技術學院', '仁愛新村'],
 '仁愛新村': ['豫章工商', '福星里'],
 '福星里': ['仁愛新村', '鄉雲里', '中興醫院'],
 '鄉雲里': ['福星里', '後站商圈'],
 '後站商圈': ['鄉雲里', '介壽公園', '板橋外站', '民族區運路口'],
 '介壽公園': ['後站商圈', '民權路口'],
 '民權路口': ['介壽公園', '新北市政府(新府路)', '板橋外站', '復興路', '捷運三民高中站(復興路)'],
 '新北市政府(新府路)': ['民權路口', '板橋公車站', '深丘里'],
 '板橋公車站': ['新北市政府(新府路)', '新北市政府(中山路)', '萬坪公園', '中山國中'],
 '新北市政府(中山路)': ['板橋公車站', '中山漢生路口'],
 '中山漢生路口': ['新北市政府(中山路)', '西安里', '中山路口', '漢生路'],
 '西安里': ['中山漢生路口', '中山民生路口', '民生中山路口', '民生路橋', '中山路口'],
 '中山民生路口': ['西安里', '埔墘國小'],
 '埔墘國小': ['中山民生路口', '埔墘(中山路)'],
 '埔墘(中山路)': ['埔墘國小', '埔墘派出所'],
 '埔墘派出所': ['埔墘(中山路)', '光復橋', '埔墘(三民路)'],
 '光復橋': ['

* bfs find path (each stop is a node)

In [141]:
import collections

def bfs(graph, start, end):
    queue, visited = collections.deque([start]), [start]
    while queue:
        vertex = queue.popleft()
        for neighbour in graph[vertex]:
            if neighbour not in visited:
                visited.append(neighbour)
                queue.append(neighbour)
            if neighbour == end:
                return visited
    return []
                
test_graph = {'A': ['B', 'D'], 'B': ['C', 'E'], 'C': ['E', 'F'], 'D': ['G'], 'E':['D', 'F', 'G', 'H'], 'F': ['H'], 'G': ['H'], 'H': []}


bfs(test_graph, 'A', 'F')

['A', 'B', 'D', 'C', 'E', 'G', 'F']

In [142]:
bfs(graph, '國父紀念館', '捷運市政府站')

['國父紀念館',
 '仁愛光復路口',
 '市政府(市府)',
 '聯合報',
 '仁愛延吉街口',
 '觀光局',
 '市議會',
 '市民住宅',
 '捷運國父紀念館站(光復)',
 '市政府(松壽)',
 '捷運國父紀念館站(忠孝)',
 '興雅國中',
 '市政府(松高)',
 '捷運市政府站']

* Route Numbers and unique RouteID list of all county

In [125]:
import os
county_file_map = {'臺北市': 'Taipei_route.json','基隆市': 'Keelung_route.json','新北市': 'NewTaipei_route.json','連江縣': 'LienchiangCounty.json','宜蘭縣': 'YilanCounty_route.json','新竹市': 'Hsinchu_route.json','新竹縣': 'HsinchuCounty_route.json','桃園市': 'Taoyuan.json','苗栗縣': 'MiaoliCounty_route.json','臺中市': 'Taichung.json','彰化縣': 'ChanghuaCounty_route.json','南投縣': 'NantouCounty_route.json','嘉義市': 'Chiayi_route.json','嘉義縣': 'ChiayiCounty_route.json','雲林縣': 'YunlinCounty_route.json','臺南市': 'Tainan_route.json','高雄市': 'Kaohsiung.json','澎湖縣': 'PenghuCounty_route.json','金門縣': 'KinmenCounty_route.json','屏東縣': 'PingtungCounty_route.json','臺東縣': 'TaitungCounty_route.json','花蓮縣': 'HualienCounty_route.json'}
    
county_routes = {}
for file in os.listdir('Transportation_data/bus_schedule/'):
    if file == '.ipynb_checkpoints': continue
    with open('Transportation_data/bus_schedule/'+file) as fp:
        bus_file = json.load(fp)
        routes = []
        for route in bus_file:
            if route['RouteID'] not in routes:
                routes.append(route['RouteID'])
    county_routes[file] = routes

for county in county_routes:
    print("{}: {}".format(county, len(county_routes[county])))

YilanCounty_route.json: 75
NewTaipei_route.json: 604
YilanCounty.json: 68
PingtungCounty_route.json: 27
ChanghuaCounty.json: 11
YunlinCounty_route.json: 7
Taoyuan.json: 379
PingtungCounty.json: 27
HsinchuCounty.json: 22
HualienCounty.json: 5
Chiayi_route.json: 3
Tainan_route.json: 127
Taichung.json: 301
Kaohsiung.json: 319
YunlinCounty.json: 7
Keelung_route.json: 136
PenghuCounty.json: 14
Taipei_route.json: 386
ChanghuaCounty_route.json: 13
NewTaipei.json: 600
TaitungCounty.json: 1
MiaoliCounty_route.json: 1
Hsinchu_route.json: 27
Keelung.json: 71
ChiayiCounty_route.json: 8
MiaoliCounty.json: 1
ChiayiCounty.json: 7
HualienCounty_route.json: 6
Tainan.json: 112
TaitungCounty_route.json: 2
LienchiangCounty.json: 41
KinmenCounty.json: 68
KinmenCounty_route.json: 97
NantouCounty_route.json: 3
Hsinchu.json: 27
Chiayi.json: 3
PenghuCounty_route.json: 14
NantouCounty.json: 3
Taipei.json: 385
HsinchuCounty_route.json: 29
